# User-Studies 

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("Data/IN5060_fall24-Pre-study.csv")
#df.columns = df.columns.str.strip().str.replace(r'\(.*?\)', '', regex=True).str.replace(' ', '_')
#print(df.columns)
# Display all columns and expand display width
pd.set_option('display.max_columns', None)     # Show all columns
pd.set_option('display.width', 1000)           # Set the display width to fit large tables
pd.set_option('display.max_colwidth', None)    # Show full column content without truncation
pd.set_option('display.max_rows', None)        # Show all rows (remove this if there are too many rows)

print(df.head(10))

                      Timestamp Participant ID (top-right corner of the screen)  How old are you?            Gender?          What describes you the best?  How comfortable are you in expressing emotions? Have you ever played with the "drummer bot" dB before?  How much prior experience do you have with interactive musical interfaces (e.g. sensor systems, digital control devices)?  How often do you use AI tools for creativity (e.g., writing, coding, music, fine arts, food recipes, etc.)? Please indicate if you have any of the following health problems.
0   2024/11/04 5:15:10 PM GMT+1                                            test                66  Prefer not to say    Student/semi-professional musician                                                3                                                    Yes                                                                                                                          4                                                              